# Macro e file .mac .sh , Ntuple e Trees

In questo notebook parleremo brevemente di come si possono creare e usare le macro in ROOT e dei tipi di file correlati al framework.

In particolare vedremo come usando i file .mac e .sh si possono "meccanizzare" ovvero usare ripetutamente le macro create precedentemente, vedremo alcuni esempio di macro (che eseguiremo sul terminale non sul notebook) e come è possibile eseguirle ripetutamente da un unico file.

Parleremo poi di due classi ROOT più avanzate:
    - TTre
    - TNtuple
    
Solitamente derivanti dall'acquisizione dati.

**!!! importante** 

Nel file seguente alcune istruzioni (import ROOT e %%cpp) sono usate solo e soltanto per far funzionare il codice sul notebook, non vanno usate nel framework!

## Macro e file .mac e .sh

Il framework di ROOT permette di caricare ed eseguire macro, tuttavia cosa è una macro?

Le librerie di ROOT possono essere sostanzialmente usate in due modi: si possono integrare in un programma compilabile in C++ (complicato da integrare e necessita di alcuni flag in compilazione) di cui parleremo brevemente oppure si possono scrivere dei codici in formalismo C++ che vengono poi eseguiti direttamente all'interno del framework, questa seconda opzione è detta macro.

Facciamo un esempio: il file TH1toTxt.cc è un file c++ in cui è presente solamente un omonima funzione, esso non è eseguibile e compilabile con gcc (il classico compilatore C++) ma può essere eseguito comodamente all'interno del framework, analizziamolo meglio

In [1]:
import ROOT

Welcome to JupyROOT 6.14/06


In [3]:
#per prima cosa vediamo cosa si trova nel file
#!!!!questo non è root sto eseguendo nella shell
!cat TH1toTxt.cc

void TH1toTxt (TH1F *h, string outfile) {
	ofstream out (outfile);
	int n=h->GetNbinsX();
	for (int i=0; i<n; i++) out<<h->GetBinCenter(i)<<"\t"<<h->GetBinContent(i)<<endl;
}


Sostanzialmente questo codice chiede un istogramma e il nome di un file di output in cui creare le coppie di punti corrispondenti ai suoi bin (così da poterne fare poi un TGraph).

Il codice può essere comodamente eseguito all'interno del framework in due modi:
1. con l'instruzione .x si esegue direttamente la macro dando i paramentri di input a patto che il nome del file e quello della funzione principale (quella che viene eseguita, simile al main in cpp) coincidano
2. con l'istruzione .L si caricano tutte le funzioni presenti nel file e queste diventano poi eseguibili liberamente all'interno del framework

Nel notebook non è possibile eseguire questi comandi (non sono cp nativi) quindi faremo un esempio direttamente da terminale, in ogni caso il codice che bisogna usare sarà:

- root -l 100mSimK12_trigger.root -> apro il file root
- .x TH1toTxt.cc (hAll, "hAllout.txt") ->eseguo la macro su hAll

oppure nel secondo caso
- root -l 100mSimK12_trigger.root -> apro il file root
- .L TH1toTxt.cc                 ->carico la macro
- TH1toTxt(hAll, "hAllout.txt") ->eseguo la macro su hAll


Il secondo approccio risulta utile nel caso in un unico file vi siano molte macro che si vogliono eseguire indipendentemente.


Infine analizziamo un ultimo caso: in caso si voglia usare la stessa macro ripetutamente è possibile scrivere un file che contiene un'esecuzione "ripetuta" della stessa, questo si può fare con i file .mac se si vuole agire all'interno del framework o con i file .sh se si vuole agire da terminale.

Supponiamo per esempio di voler eseguire ripetutamente la macro di prima sui TH1F hSim, hAll, hRec ed hSel allora è possibile "meccanizzare" la cosa in due modi come vedremo nei due file TH1conv.mac e TH1conv.sh

In [4]:
!cat TH1conv.mac

.x TH1toTxt.cc (hSim, "hSimout.txt")
.x TH1toTxt.cc (hAll, "hAllout.txt")
.x TH1toTxt.cc (hSel, "hSelout.txt")
.x TH1toTxt.cc (hRec, "hRecout.txt")


In questo file vi è semplicemente l'istruzione di prima ripetuta 4 volte questo file va eseguito sempre con .x all'interno del framework di root quindi si può eseguire anche all'interno di un file già aperto.


In alternativa se all'interno della macro vi sono le istruzioni necessarie per ricevere autonomamente i dati (ovvero se per eseguirla non è necessario essere già all'interno di uno specifico TFile allora è possibile eseguire la macro direttamente nella shell del terminale e la sua "meccanizzazione" può essere eseguita anche con un file .sh nel seguente modo: creiamo una macro che disegna e salva i file txt creati in precedenza:

In [5]:
!cat TxttoTGraph.cc

void TxttoTGraph (string name) {
	TGraph * gr= new TGraph((name+".txt").c_str());
	TCanvas *c= new TCanvas("c", "c", 600, 600);
	c->cd();
	gr->Draw();
	c->Draw();
	c->Print((name+".png").c_str());
}

//to execute in batch
//root -l -b 'TxttoTGraph.cc ("hAllout"); exit (0)'


Il file semplicemente crea un TGraph e poi salva il canvas in png, piccola nota è il fatto che il metodo print chiede un array di char quindi è necessario convertire con .c_str().

La macro in questo caso può essere eseguita dentro al framework come già visto semplicemente chiamando 
- .x TxttoTGraph.cc ("hAllout")

Oppure eseguita nella shell con l'istruzione
- root -l -b 'TxttoTGraph.cc ("hAllout"); exit (0)'
Dove exit zero serve per tornare poi al terminale e -b serve per eseguire in background (ovvero non mostrerà i canvas)

Per meccanizzare questa operazione ai quattro file creati prima basta scrivere il codice di esecuzione in un file .mac da eseguire nel framework (quindi con quattro copie della prima istruzione) oppure in modo da eseguirlo all'esterno in un file sh eseguibile come un normale file bash

In [6]:
!cat Txtconv.sh

root -l -b 'TxttoTGraph.cc ("hAllout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hSimout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hRecout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hSelout"); exit (0)'


Questo file semplicemente esegue 4 volte l'struzione ed esce ogni volta in modo che sia nuovamente chiamabile da shell, per eseguirlo è sufficiente digitare
    - source Txtconv.sh

La cosa comoda è che volendo in questo secondo modo si possono aggiungere comandi nativi della shell, per esempio si può chiedere alla funzione di aprire i file .png creati, come in questo secondo file (analogo al primo ma con un'istruzione in più):

In [7]:
!cat Txtconv2.sh

root -l -b 'TxttoTGraph.cc ("hAllout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hSimout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hRecout"); exit (0)'
root -l -b 'TxttoTGraph.cc ("hSelout"); exit (0)'

eog *.png &


## TNtuple

Le [TNtuple](https://root.cern.ch/doc/master/classTNtuple.html) sono per definizione "A simple TTree restricted to a list of float variables only."

Affronteremo più avanti i TTree ma in generale non vi è molta diffrenza tra le due classi, una Ntupla sostanzialmente permette di immagazzinare una serie di dati mantenendoli nell'ordine di aquisizione.

La funzionalità maggiore è la possibilità di plottare i dati in coppia in modo tale da cercare correlazioni e di mantenere in un unico oggetto tutti i dati su cui si vorrà poi fare analisi statistica ( supponendo essi derivino da un'acuisizione comune).

In generale non ci addentreremo molto nel come creare e modificare Ntuple e TTre poichè questa cosa viene in generale fatti dal software di acquisizione ma piuttosto su come usare i dati contenuti.